In [189]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import networkx as nx
import community as community_louvain
# import holoviews as hv
# from holoviews import opts
# hv.extension('bokeh')
import bokeh as bk
from bokeh import plotting as bplt
from bokeh import models as bmod
from bokeh import io as bkio
from bokeh.models import CustomJS
from faker import Faker
import networkx as nx
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio

import sys
sys.path.append('../')
from helper.commonHelper import calculate_angle

In [190]:
pd.set_option('display.max_columns', None)
path = '../data/HI-Small_Trans.csv'
# path = '/media/data2/drop/HI-Small_Trans.csv'
df = pd.read_csv(path)

In [191]:
df.head(10)

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0
5,2022/09/01 00:03,1,8000F5AD0,1,8000F5AD0,6162.44,US Dollar,6162.44,US Dollar,Reinvestment,0
6,2022/09/01 00:08,1,8000EBAC0,1,8000EBAC0,14.26,US Dollar,14.26,US Dollar,Reinvestment,0
7,2022/09/01 00:16,1,8000EC1E0,1,8000EC1E0,11.86,US Dollar,11.86,US Dollar,Reinvestment,0
8,2022/09/01 00:26,12,8000EC280,2439,8017BF800,7.66,US Dollar,7.66,US Dollar,Credit Card,0
9,2022/09/01 00:21,1,8000EDEC0,211050,80AEF5310,383.71,US Dollar,383.71,US Dollar,Credit Card,0


In [192]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078345 entries, 0 to 5078344
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Timestamp           object 
 1   From Bank           int64  
 2   Account             object 
 3   To Bank             int64  
 4   Account.1           object 
 5   Amount Received     float64
 6   Receiving Currency  object 
 7   Amount Paid         float64
 8   Payment Currency    object 
 9   Payment Format      object 
 10  Is Laundering       int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 426.2+ MB


In [193]:
df['id_from'] = df['From Bank'].astype(str) + '_' + df['Account']
df['id_to'] = df['To Bank'].astype(str) + '_' + df['Account.1']

In [194]:
df['id_from'].nunique(), df['id_to'].nunique()

(496999, 420640)

In [195]:
unique_values = pd.concat([df['id_from'], df['id_to']])
df_node = pd.DataFrame(unique_values.unique(), columns=['ID'])
df_node.head(10)

,ID
0,10_8000EBD30
1,3208_8000F4580
2,3209_8000F4670
3,12_8000F5030
4,10_8000F5200
5,1_8000F5AD0
6,1_8000EBAC0
7,1_8000EC1E0
8,12_8000EC280
9,1_8000EDEC0


In [196]:
df_edge = df.drop(columns=[
    'From Bank',
    'Account',
    'To Bank',
    'Account.1'
])

df_edge.head(10)

,Timestamp,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,id_from,id_to
0,2022/09/01 00:20,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0,10_8000EBD30,10_8000EBD30
1,2022/09/01 00:20,0.01,US Dollar,0.01,US Dollar,Cheque,0,3208_8000F4580,1_8000F5340
2,2022/09/01 00:00,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0,3209_8000F4670,3209_8000F4670
3,2022/09/01 00:02,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0,12_8000F5030,12_8000F5030
4,2022/09/01 00:06,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0,10_8000F5200,10_8000F5200
5,2022/09/01 00:03,6162.44,US Dollar,6162.44,US Dollar,Reinvestment,0,1_8000F5AD0,1_8000F5AD0
6,2022/09/01 00:08,14.26,US Dollar,14.26,US Dollar,Reinvestment,0,1_8000EBAC0,1_8000EBAC0
7,2022/09/01 00:16,11.86,US Dollar,11.86,US Dollar,Reinvestment,0,1_8000EC1E0,1_8000EC1E0
8,2022/09/01 00:26,7.66,US Dollar,7.66,US Dollar,Credit Card,0,12_8000EC280,2439_8017BF800
9,2022/09/01 00:21,383.71,US Dollar,383.71,US Dollar,Credit Card,0,1_8000EDEC0,211050_80AEF5310


In [197]:
# from jcopml.plot import plot_missing_value
# plot_missing_value(df_edge[df_edge['Is Laundering']==1])
df_edge[df_edge['Is Laundering']==1][['id_from']].drop_duplicates()

,id_from
4742,70_100428660
112579,231004_812EE86F0
124955,70_1004286A8
137520,21174_800737690
216900,70_100428738
...,...
5066067,110_80D580290
5067270,2845_8079CDF20
5071168,111433_8065C2380
5073375,9_80727F930


In [198]:
df_edge[df_edge['Is Laundering']==1][['id_to']].drop_duplicates()

,id_to
4742,1124_800825340
40621,11474_805B716C0
69152,15980_80B39E7B0
85763,113798_80DC756E0
90684,32375_80E480620
...,...
5070174,139298_80EE01F40
5070832,1467_8016FDCE0
5071168,21550_8065C69E0
5073375,219557_8072A28C0


In [199]:
df_edge.columns

Index(['Timestamp', 'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering', 'id_from',
       'id_to'],
      dtype='object')

In [200]:
# Membuat graph dari DataFrame
G = nx.from_pandas_edgelist(
    df_edge[df_edge['Is Laundering']==1], 'id_from', 'id_to', 
    [
        'Amount Received', 'Receiving Currency', 'Amount Paid',
        'Payment Currency', 'Payment Format', 'Is Laundering',
        'id_from', 'id_to', 
    ],
    create_using= nx.MultiGraph #nx.MultiDiGraph() kalau berarah
)

# G = nx.karate_club_graph()

# node attribute
# tidak ada selain id

In [201]:
# tambah atribut node
fake = Faker()
df_node['Nama'] = [fake.name() for _ in range(len(df_node))]
df_node['ID_index'] = df_node['ID']
node_attributes = df_node.set_index('ID_index').to_dict('index')
nx.set_node_attributes(G, node_attributes)

G.nodes(data=True)

NodeDataView({'70_100428660': {'ID': '70_100428660', 'Nama': 'Seth Adams'}, '1124_800825340': {'ID': '1124_800825340', 'Nama': 'Renee Mckee'}, '11474_805B716C0': {'ID': '11474_805B716C0', 'Nama': 'Michael Webb'}, '15980_80B39E7B0': {'ID': '15980_80B39E7B0', 'Nama': 'Matthew Salinas'}, '113798_80DC756E0': {'ID': '113798_80DC756E0', 'Nama': 'Todd Andersen'}, '32375_80E480620': {'ID': '32375_80E480620', 'Nama': 'Melissa Atkins'}, '231004_812EE86F0': {'ID': '231004_812EE86F0', 'Nama': 'Cheryl Martinez'}, '21940_812EE89C0': {'ID': '21940_812EE89C0', 'Nama': 'Angela Ali'}, '152980_8140702D0': {'ID': '152980_8140702D0', 'Nama': 'Cynthia Ford'}, '70_1004286A8': {'ID': '70_1004286A8', 'Nama': 'Lindsey Norris'}, '2591_800926A30': {'ID': '2591_800926A30', 'Nama': 'Marcus Woodward'}, '21174_800737690': {'ID': '21174_800737690', 'Nama': 'Christie Rodriguez'}, '12_80011F990': {'ID': '12_80011F990', 'Nama': 'Jennifer James'}, '20_80020C5B0': {'ID': '20_80020C5B0', 'Nama': 'Matthew Brown'}, '20_80006A

In [202]:
# community detection

# from networkx.algorithms.community import girvan_newman
# # Langkah 1: Gunakan algoritma Girvan-Newman untuk mendeteksi komunitas
# comp = girvan_newman(G)

# # Langkah 2: Ambil pembagian komunitas sampai level maksimal (hingga node-node tidak saling terhubung)
# # Mendapatkan semua level pembagian
# for i in range(3):
#     communities = next(comp)
# # for community in comp:
# #     communities = community

# # Langkah 3: Buat dataframe dari komunitas yang dihasilkan
# community_series = pd.Series(communities).reset_index()
# community_series = community_series.explode(0).reset_index(drop=True)
# community_series.columns = ['partition', 'node']

# # Langkah 4: Set node attributes tanpa loop
# partition_dict = community_series.set_index('node')['partition'].to_dict()
# partition_dict
# nx.set_node_attributes(G, partition_dict, 'partition')

# =======================================================

# # Langkah 1: Gunakan algoritma Louvain untuk mendeteksi komunitas
# partition = community_louvain.best_partition(G)
# # Langkah 2: Tambahkan atribut "partition" pada setiap node
# nx.set_node_attributes(G, partition, 'partition')


# =======================================================

# # Langkah 1: Gunakan algoritma Louvain untuk mendeteksi komunitas
# partition = community_louvain.best_partition(G)

# # Langkah 2: Buat dataframe dari partition
# partition_df = pd.DataFrame(list(partition.items()), columns=['node', 'partition'])

# # Langkah 3: Hitung jumlah node dalam setiap partisi
# partition_counts = partition_df['partition'].value_counts().reset_index()
# partition_counts.columns = ['partition', 'count']

# # Langkah 4: Urutkan partisi berdasarkan jumlah node (dari terbesar ke terkecil)
# partition_counts['sorted_partition'] = partition_counts['count'].rank(method='first', ascending=False).astype(int) - 1
# partition_map = partition_counts.set_index('partition')['sorted_partition'].to_dict()

# # Langkah 5: Update dataframe dengan partisi yang sudah diurutkan
# partition_df['sorted_partition'] = partition_df['partition'].map(partition_map)

# # Langkah 6: Tambahkan atribut "partition" yang diurutkan pada setiap node
# nx.set_node_attributes(G, partition_df.set_index('node')['sorted_partition'].to_dict(), 'partition')

# =======================================================

# Langkah 1: Deteksi komponen terhubung
components = list(nx.connected_components(G))

# Langkah 2: Urutkan komponen berdasarkan jumlah node (dari terbesar ke terkecil)
components_sorted = sorted(components, key=len, reverse=True)

# Langkah 3: Buat dataframe untuk komponen
component_df = pd.DataFrame([(node, i) for i, comp in enumerate(components_sorted) for node in comp], columns=['node', 'partition'])

# Langkah 4: Tetapkan atribut "partition" pada setiap node
partition_dict = component_df.set_index('node')['partition'].to_dict()
nx.set_node_attributes(G, partition_dict, 'partition')

In [203]:
# from pyvis.network import Network
# nt = Network('500px', '500px')
# nt.from_nx(G) #nx.karate_club_graph()
# nt.show_buttons(filter_=['physics'])
# nt.write_html('ini.html')

In [204]:
for edge in G.nodes(data=True):
    print(edge[1])
    if 1==1:
        break

{'ID': '70_100428660', 'Nama': 'Seth Adams', 'partition': 0}


In [205]:
G.nodes

NodeView(('70_100428660', '1124_800825340', '11474_805B716C0', '15980_80B39E7B0', '113798_80DC756E0', '32375_80E480620', '231004_812EE86F0', '21940_812EE89C0', '152980_8140702D0', '70_1004286A8', '2591_800926A30', '21174_800737690', '12_80011F990', '20_80020C5B0', '20_80006A5E0', '220_8007A5B70', '1244_80093C0D0', '513_80078E200', '20_80066B990', '410_8002CC310', '1292_8004030A0', '1601_800578800', '1_800AAF0B0', '1420_8006FB970', '10_800449990', '11852_800AEBA90', '10_800692550', '11056_8007486D0', '24144_80D9B69A0', '70_100428738', '10656_8044072A0', '70_100428858', '8_8090A6540', '70_100428978', '141312_80FC59DE0', '1467_8013C4030', '20_80BC62F10', '240229_80F025640', '217_80FD27570', '24856_8090E8EB0', '71_804ABCE90', '213737_805494C30', '14290_801B949C0', '10057_803DE1580', '28628_80ACEE280', '119_811C597B0', '48309_811C599A0', '150240_812D22980', '222_811B83280', '8805_805B66DC0', '143508_8127331A0', '250508_8127332E0', '112354_8072445F0', '21387_811B12170', '21387_811B121C0', '1

In [206]:
# Membuat color random untuk grup
# Menentukan colorscale
partition = component_df.partition.unique()
colorscale = px.colors.sequential.Redor_r  # _r dibalik colorscale
length = len(partition)
# color_range = np.random.rand(length)
color_range = np.linspace(0, 1, length)
# Menghasilkan warna acak untuk setiap node
node_colors = px.colors.sample_colorscale(colorscale, color_range)
# Membuat dictionary dengan array_key sebagai key dan node_colors sebagai value
color_dict = dict(zip(
    partition, 
    node_colors
))

# pake plotly

In [207]:
# Mendapatkan posisi node
pos = nx.nx_agraph.graphviz_layout(G, prog="neato", args="")

# Membuat edge trace
edge_x = []
edge_y = []
# Menyusun data untuk titik tengah
mid_x = []
mid_y = []
mid_text = []
mid_angle = []

for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

    # Menyusun teks hover untuk setiap edge
    # edge_info = f"From: {edge[0]}<br>To: {edge[1]}"
    # edge_info = ""
    # for attr, value in edge[2].items():
    #     edge_info += f"<br>{attr}: {value}"

    edge_info = edge[2]

    # Menyusun data untuk titik tengah
    mid_x.append((x0 + x1) / 2)
    mid_y.append((y0 + y1) / 2)
    mid_text.append(edge_info)
    mid_angle.append(calculate_angle(x0, y0, x1, y1))

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    text=None,
    hoverinfo="text",
    line=dict(width=1.5, color='#888'),
    mode='lines',
)

# Menyusun trace untuk titik tengah
mid_trace = go.Scatter(
    name="Edge",
    x=mid_x, y=mid_y,
    mode='markers',
    marker=dict(
        size=6, color='Blue',
        symbol='square-x',
        angle=mid_angle,
    ),
    customdata=pd.DataFrame(mid_text),
    hovertemplate =
    "<b>Edge</b><br><br>"+
    "From: <b>%{customdata[6]}</b><br>" +
    "Amount Paid: %{customdata[2]:,.0f}<br>" +  # Memformat dengan pemisah ribuan
    "Payment Currency: %{customdata[3]}<br>" +
    "<br>"+
    "To: <b>%{customdata[7]}</b><br>" +
    "Amount Received: %{customdata[0]:,.0f}<br>" +  # Memformat dengan pemisah ribuan
    "Receiving Currency: %{customdata[1]}<br>" +
    "<br>"+
    "Payment Format: %{customdata[4]}<br>" +
    "Is Laundering: %{customdata[5]}<br>",
)


# Membuat node trace
node_x = []
node_y = []
node_colors = []
node_text = []

for node in G.nodes(data=True):
    x, y = pos[node[0]]
    node_x.append(x)
    node_y.append(y)
    node_colors.append(
        color_dict[node[1]['partition']]
    )  # Menambahkan warna berdasarkan partisi
    
    # Menyusun teks hover untuk setiap edge
    node_info = ""
    for attr, value in node[1].items():
        node_info += f"<br>{attr}: {value}"
    node_text.append(node_info)

node_trace = go.Scatter(
    name="Node",
    x=node_x, y=node_y,
    mode='markers',
    marker=dict(
        # reversescale=False,
        # cmax=max(node_colors),
        # cmin=min(node_colors),
        color=list(map(str, node_colors)), #konversi spya jadi warna diskrit
        size=10,
        line_width=2,
        colorscale="Viridis"
    ),
    text=node_text,
    hovertemplate="<b>Node</b><br>"+"%{text}",
)

# Membuat figure dan plot
fig = go.Figure(
    data=[edge_trace, mid_trace, node_trace],
    layout=go.Layout(
        # title='Money Laundering Graph',
        titlefont=dict(size=16),
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
    )
)

# Menyimpan visualisasi ke file HTML
pio.write_html(
    fig,
    file='graph_plotly.html',
    auto_open=False,
    # ini untuk jinja
    full_html=False,
    include_plotlyjs=False,
)

In [208]:
import importlib
importlib.reload(sys.modules['helper.PlotlyFigHelpers'])
from helper.PlotlyFigHelpers import fig_bar, fig_line, fig_pie, card_template_html_string

# len(df_node),len(df_edge)
len(G.nodes(data=True)),len(G.edges(data=True))

(6357, 5177)

In [209]:
df_node_partisi = df_node.merge(
    component_df, left_on="ID", right_on="node", how="inner"
).drop(columns=["node","ID_index"])

In [210]:
df_edge[df_edge['Is Laundering']==1].merge(df_node_partisi, left_on='id_from', right_on='ID').drop(columns=['id_to', 'id_from'])

,Timestamp,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,ID,Nama,partition
0,2022/09/01 00:21,389769.39,US Dollar,389769.39,US Dollar,Cheque,1,70_100428660,Seth Adams,0
1,2022/09/01 00:03,29024.33,US Dollar,29024.33,US Dollar,Credit Card,1,70_100428660,Seth Adams,0
2,2022/09/01 00:01,792.92,US Dollar,792.92,US Dollar,Credit Card,1,70_100428660,Seth Adams,0
3,2022/09/01 00:03,13171425.53,US Dollar,13171425.53,US Dollar,Cheque,1,70_100428660,Seth Adams,0
4,2022/09/01 00:23,14288.83,US Dollar,14288.83,US Dollar,Cash,1,70_100428660,Seth Adams,0
...,...,...,...,...,...,...,...,...,...,...
5172,2022/09/10 23:29,7773.42,US Dollar,7773.42,US Dollar,ACH,1,111433_8065C2380,Brian Gill,1840
5173,2022/09/10 23:07,448920.65,Ruble,448920.65,Ruble,Cheque,1,9_80727F930,Tiffany Webb,1841
5174,2022/09/12 03:34,20085.71,Saudi Riyal,20085.71,Saudi Riyal,ACH,1,222_812D127D0,Krista Torres,834
5175,2022/09/10 23:43,3314.36,US Dollar,3314.36,US Dollar,ACH,1,17327_8095F6C20,Brent Warren,298


In [211]:
# file='bar_degree_from.html',
bar = fig_bar(
    df_edge[df_edge['Is Laundering']==1].merge(df_node_partisi, left_on='id_from', right_on='ID',how='left').drop(columns=['id_to', 'id_from']),
    "Nama",
    "Timestamp",
    "count",
    xlabel="Jumlah Transaksi",
    added_agg_data={"ID":"first","partition":"first"},
    color_by_filter=True,
)
pio.write_html(
    bar,
    file='bar_degree_from.html',
    auto_open=False,
    # ini untuk jinja
    full_html=False,
    include_plotlyjs=False,
)

In [212]:
# file='bar_degree_to.html',
bar = fig_bar(
    df_edge[df_edge['Is Laundering']==1].merge(df_node_partisi, left_on='id_to', right_on='ID',how='left').drop(columns=['id_to', 'id_from']),
    "Nama",
    "Timestamp",
    "count",
    xlabel="Jumlah Transaksi",
    added_agg_data={"ID":"first","partition":"first"},
    color_by_filter=True,
)
pio.write_html(
    bar,
    file='bar_degree_to.html',
    auto_open=False,
    # ini untuk jinja
    full_html=False,
    include_plotlyjs=False,
)

In [213]:
# "Payment Format",
bar = fig_bar(
    df_edge[df_edge['Is Laundering']==1],
    "Payment Format",
    "Timestamp",
    "count",
    xlabel="Jumlah Transaksi",
    color_by_filter=True,
)
pio.write_html(
    bar,
    file='bar_payment_format.html',
    auto_open=False,
    # ini untuk jinja
    full_html=False,
    include_plotlyjs=False,
)

In [214]:
#   "Receiving Currency",
bar = fig_bar(
    df_edge[df_edge['Is Laundering']==1],
    "Receiving Currency",
    "Timestamp",
    "count",
    xlabel="Jumlah Transaksi",
    color_by_filter=True,
)
pio.write_html(
    bar,
    file='bar_receiving_currency.html',
    auto_open=False,
    # ini untuk jinja
    full_html=False,
    include_plotlyjs=False,
)

In [215]:
#   "Payment Currency",
bar = fig_bar(
    df_edge[df_edge['Is Laundering']==1],
    "Payment Currency",
    "Timestamp",
    "count",
    xlabel="Jumlah Transaksi",
    color_by_filter=True,
)
pio.write_html(
    bar,
    file='bar_payment_currency.html',
    auto_open=False,
    # ini untuk jinja
    full_html=False,
    include_plotlyjs=False,
)

In [216]:
df_edge['Timestamp'] = df_edge['Timestamp'].astype('datetime64[ns]')
df_edge['date_only'] = df_edge['Timestamp'].dt.date
df_edge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078345 entries, 0 to 5078344
Data columns (total 10 columns):
 #   Column              Dtype         
---  ------              -----         
 0   Timestamp           datetime64[ns]
 1   Amount Received     float64       
 2   Receiving Currency  object        
 3   Amount Paid         float64       
 4   Payment Currency    object        
 5   Payment Format      object        
 6   Is Laundering       int64         
 7   id_from             object        
 8   id_to               object        
 9   date_only           object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 387.4+ MB


In [224]:
# line
ini = df_edge[df_edge['Is Laundering']==1].groupby(by=["date_only"]).agg({'Is Laundering':'count'}).rename(columns={'Is Laundering':'Jumlah'}).reset_index()

fig = px.line(
        ini, x='date_only', y='Jumlah', #color='tahun_bulan', 
        # title='Jumlah transaksi di MAKASSAR dalam bulan tertentu di tahun 2023',
        # labels={'tahun_bulan': 'Bulan', 'count': 'Jumlah'},
        markers=True,  # Menambahkan marker untuk tiap titik data
        template='seaborn'  # Anda dapat mengganti dengan template yang diinginkan
    )

pio.write_html(
    fig,
    file='line_transaction_date.html',
    auto_open=False,
    # ini untuk jinja
    full_html=False,
    include_plotlyjs=False,
)

In [233]:
# line
ini = df_node_partisi.groupby('partition').agg({'ID': 'nunique'}).sort_values('ID', ascending=False).reset_index()
ini.rename(columns={'ID': 'Jumlah Anggota'}, inplace=True)

fig = px.line(
        ini, x='partition', y='Jumlah Anggota', #color='tahun_bulan', 
        # title='Jumlah transaksi di MAKASSAR dalam bulan tertentu di tahun 2023',
        # labels={'tahun_bulan': 'Bulan', 'count': 'Jumlah'},
        markers=True,  # Menambahkan marker untuk tiap titik data
        template='seaborn'  # Anda dapat mengganti dengan template yang diinginkan
    )

pio.write_html(
    fig,
    file='line_partition.html',
    auto_open=False,
    # ini untuk jinja
    full_html=False,
    include_plotlyjs=False,
)

# pake Bokeh

In [ ]:
# Membuat Graph
# pos = nx.nx_agraph.graphviz_layout(G, prog="neato", args="")
pos = nx.spring_layout
g = bplt.from_networkx(G, pos, scale=1, center=(0, 0))

# Membuat plot figure
p = bmod.Plot(
    width=800, height=800,
    x_range=bmod.Range1d(-1.1, 1.1), 
    y_range=bmod.Range1d(-1.1, 1.1),
    toolbar_location="above"
)

# Menambahkan tools interaktif termasuk alat zoom
p.add_tools(
    bmod.HoverTool(tooltips=[
        # ("index", "$index"), 
        ("Account From", "@{id_from}"),
        ("Account To", "@{id_to}"),
        ("Amount Paid", "@{Amount Paid}"),
        ("Receiving Currency", "@{Receiving Currency}"),
        ("Amount Received", "@{Amount Received}"),
        ("Payment Currency", "@{Payment Currency}"),
        ("Payment Format", "@{Payment Format}"),
        ("Is Laundering", "@{Is Laundering}"),
    ]), 
    'reset',
    'pan',
    'box_zoom',
    'wheel_zoom',
    'save',
    bmod.TapTool(), 
    bmod.BoxSelectTool(),
    bmod.PointDrawTool(renderers=[g.node_renderer])  # Menambahkan PointDrawTool
)
p.toolbar.active_scroll = p.select_one(bmod.WheelZoomTool)

# Menambahkan styling
initial_radius = 0.01
g.node_renderer.glyph = bmod.Circle(radius=initial_radius, fill_color='skyblue')
g.node_renderer.hover_glyph = bmod.Circle(radius=initial_radius, fill_color='#24b06c')
g.node_renderer.selection_glyph = bmod.Circle(radius=initial_radius, fill_color='#b09b24')
g.node_renderer.nonselection_glyph = bmod.Circle(radius=initial_radius, fill_color='skyblue')

g.edge_renderer.glyph = bmod.MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=1)
g.edge_renderer.hover_glyph = bmod.MultiLine(line_color="#24b06c", line_alpha=0.8, line_width=1)
g.edge_renderer.selection_glyph = bmod.MultiLine(line_color="#b09b24", line_alpha=0.8, line_width=1)
g.edge_renderer.nonselection_glyph = bmod.MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=1)

g.inspection_policy = bmod.graphs.EdgesAndLinkedNodes() 
g.selection_policy = bmod.graphs.EdgesAndLinkedNodes()
# NodesAndLinkedEdges()
#or EdgesAndLinkedNodes()
#NodesAndAdjacentNodes()

# Append graph to plot 
p.renderers.append(g)

# CustomJS callback untuk mengubah ukuran node berdasarkan zoom level
callback = CustomJS(args=dict(g=g, p=p), code="""
    var x_range = p.x_range;
    var y_range = p.y_range;
    var x_span = x_range.end - x_range.start;
    var y_span = y_range.end - y_range.start;
    var zoom_level = Math.max(x_span, y_span);
    var new_radius = zoom_level / 220;

    g.node_renderer.glyph.radius = {value: new_radius};
    g.node_renderer.hover_glyph.radius = {value: new_radius};
    g.node_renderer.selection_glyph.radius = {value: new_radius};
    g.node_renderer.nonselection_glyph.radius = {value: new_radius};
""")

# Menambahkan callback ke x_range dan y_range
p.x_range.js_on_change('start', callback)
p.x_range.js_on_change('end', callback)
p.y_range.js_on_change('start', callback)
p.y_range.js_on_change('end', callback)

# Menyimpan plot ke file HTML
bkio.output_file("graph.html")
bkio.save(p)

# Menampilkan plot
# bk.io.show(p)

In [ ]:

len(g.edge_renderer.data_source.data['Payment Currency'])
# len(grp.edges())
# df_edge[df_edge['Is Laundering']==1].shape

# sum all

# Bebas

In [ ]:
cek = df.groupby(['id_to']).agg({
    'id_from':'nunique',
    'Is Laundering':'first',
    'Amount Paid':'sum',
}).reset_index().sort_values(['Amount Paid'], ascending=False)

cek

In [ ]:
def find_parameters_and_result(numbers):
    length = len(numbers)
    for i in range(length):
        sum_rest = sum(numbers) - numbers[i]
        if sum_rest == numbers[i]:
            result = numbers[i]
            parameters = numbers[:i] + numbers[i+1:]
            return parameters, result
    return None, None

# Contoh penggunaan
numbers = [19, 9, 10]
parameters, result = find_parameters_and_result(numbers)
print("Parameters:", parameters)
print("Result:", result)

In [ ]:

df['Group'] = df.groupby('Timestamp').ngroup()
df_sorted = df.sort_values(by='Group')
df_sorted[
    (df_sorted['Group']==3) & 
    (df_sorted['Is Laundering']==1)
]
